# Dependências

In [157]:
!pipenv install pandas
!pipenv install networkx pyvis

Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
Installing pandas...
Resolving pandas...
✔ Installation Succeeded
⠋ Installing pandas...
Installing dependencies from Pipfile.lock (d5e100)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
Installing networkx...
Resolving networkx...
✔ Installation 

# Construção da Rede

In [1]:
import pandas as pd
import networkx as nx
from collections import Counter
from pyvis.network import Network
import re

In [2]:
# --- CONFIGURAÇÕES ---
INPUT_FILE = 'enriched_cause_effect_relations.csv'
OUTPUT_VISUAL = 'bayesian_network.html'
OUTPUT_FILE = 'bayesian_network.csv'

LIMIAR_FREQUENCY = 1

In [3]:
STOP_WORDS_CUSTOM = {
    # Lixo comum de PDF/Extração
    "tcc", "artigo", "original", "referências", "bibliográficas", "issn", "doi", 
    "página", "conclusões", "resultados", "objetivo", "método", "figura", "tabela",
    "et al", "apud", "nº", "https", "www", "copyright", "licença",
    
    # Stop words de outros idiomas que aparecem com frequência
    "the", "of", "and", "in", "to", "is", "a", "with", "for", "by", "that", "it",
    "as", "on", "are", "from", "or", "an", "may", "can", "have", "been",
    "la", "el", "de", "en", "y", "que", "un", "una", "los", "las",
    
    # Termos muito genéricos que não são úteis como nós
    "estudo", "pesquisa", "autor", "paciente", "indivíduo", "grupo"
}

In [4]:
# CONSOLIDAÇÃO FINAL DOS NÓS (Normalização)
#    Esta etapa mapeia variações para um nome canônico.
#    Ex: "HAS" e "Hipertensão Arterial" -> "Hipertensão"
def normalize_node_name(entity_text):
  if not isinstance(entity_text, str):
    return None
  text = entity_text.lower().strip()
  text = re.sub(r"^[.,'\"“‘\[\(]+|[.,'\"”’\]\)]+$", "", text.strip())

  # Regras de normalização
  # Categoria: Pressão Arterial
  if 'pressão arterial' in text or 'pa' in text or 'pas' in text or 'pad' in text or 'pressóricos' in text:
    return 'Pressão Arterial (PA)'
  if 'hipertensão' in text or 'has' in text or 'hipertensivo' in text:
    return 'Hipertensão Arterial (HAS)'

	# Categoria: Doenças e Condições
  if 'cardiovascular' in text or 'dvc' in text or 'coronariana' in text or 'infarto' in text or 'avc' in text:
    return 'Doenças Cardiovasculares (DVC)'
  if 'doença renal' in text or 'drc' in text:
    return 'Doença Renal Crônica (DRC)'
  if 'diabetes' in text or 'glicemia' in text or 'insulina' in text:
    return 'Diabetes / Glicemia'
  if 'doença crônica' in text:
    return 'Doença Crônica'
  if 'obesidade' in text or 'excesso de peso' in text or 'sobrepeso' in text:
    return 'Obesidade / Sobrepeso'
  if 'tabagismo' in text or 'fumo' in text or 'fumar' in text or 'cigarro' in text:
    return 'Tabagismo'
  if 'alcoolismo' in text or 'álcool' in text or 'alcoólicas' in text:
    return 'Consumo de Álcool'
  if 'sedentarismo' in text or 'atividade física' in text or 'exercício' in text:
    return 'Atividade Física / Sedentarismo'
  if 'sal' in text or 'sódio' in text:
    return 'Consumo de Sal / Sódio'
  if 'dieta' in text or 'alimentar' in text or 'nutrição' in text:
    return 'Dieta / Alimentação'
  if 'estresse' in text:
    return 'Estresse'
  if 'dislipidemia' in text or 'colesterol' in text or 'triglicerídeos' in text:
    return 'Dislipidemia / Colesterol'
  if 'potássio' in text:
    return 'Consumo de Potássio'
  if 'risco' in text:
    return 'Fator de Risco'

  cleared_text = re.sub(r'^(o|a|os|as)\s+', '', text)
  if len(cleared_text) > 4: return None
  if cleared_text in STOP_WORDS_CUSTOM: return None

  return cleared_text.capitalize()

In [5]:
# 1. LER O CSV DE RELAÇÕES
try:
    df = pd.read_csv(INPUT_FILE)
except FileNotFoundError:
    print(f"ERRO: Arquivo '{INPUT_FILE}' não encontrado.")
    exit()

print(f"Lidas {len(df)} relações do arquivo.")

Lidas 728 relações do arquivo.


In [6]:
# 2. NORMALIZAR AS ENTIDADES E FILTRAR ARESTAS
edge_data = []
for index, row in df.iterrows():
  cause = row.get('causa')
  effect = row.get('efeito')
  direction = row.get('direcao')
  
  if direction == 'Efeito <- Causa':
    real_cause = effect
    real_effect = cause
  elif direction == 'Causa -> Efeito':
    real_cause = cause
    real_effect = effect
  
  cannonical_cause = normalize_node_name(real_cause)
  cannonical_effect = normalize_node_name(real_effect)

  if cannonical_cause and cannonical_effect and cannonical_cause != cannonical_effect:
    edge_data.append({
      'No_Causa': cannonical_cause,
      'No_Efeito': cannonical_effect,
      'Direcao': direction,
      'Causa_Original': cause,
      'Efeito_Original': effect,
      'Score_Relacao': row.get('score_relacao', 0),
      'Arquivo_Origem': row.get('arquivo_origem', 'N/A'),
      'Sentenca_Original': row.get('sentenca_original', 'N/A')
    })

df_normalized = pd.DataFrame(edge_data)
print(f"Após normalização, {len(df_normalized)} relações válidas encontradas.")

Após normalização, 92 relações válidas encontradas.


In [8]:
# 3. SALVAR O DATAFRAME NORMALIZADO
if not df_normalized.empty:
  df_normalized.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
  print(f"INFO: Normalized relations saved to '{OUTPUT_FILE}'.")
  print("INFO: Displaying the first 10 normalized relations:")
  print(df_normalized.head(10))
else:
  print("INFO: No valid relations found after normalization. Exiting script.")
  exit()

INFO: Normalized relations saved to 'bayesian_network.csv'.
INFO: Displaying the first 10 normalized relations:
                No_Causa               No_Efeito          Direcao  \
0                    Ame          Fator de Risco  Causa -> Efeito   
1                    Sra   Pressão Arterial (PA)  Causa -> Efeito   
2  Pressão Arterial (PA)  Consumo de Sal / Sódio  Causa -> Efeito   
3                    Sna   Pressão Arterial (PA)  Causa -> Efeito   
4  Pressão Arterial (PA)          Fator de Risco  Causa -> Efeito   
5  Pressão Arterial (PA)     Diabetes / Glicemia  Causa -> Efeito   
6      Consumo de Álcool   Pressão Arterial (PA)  Causa -> Efeito   
7  Pressão Arterial (PA)     Diabetes / Glicemia  Causa -> Efeito   
8                     Ri          Fator de Risco  Causa -> Efeito   
9  Pressão Arterial (PA)  Consumo de Sal / Sódio  Causa -> Efeito   

                                      Causa_Original  \
0                                              a AME   
1               

In [8]:
# 4. CONSTRUÇÃO DA ESTRUTURA DA REDE (DAG)
# Contar a frequência de cada aresta
edge_list = list(zip(df_normalized['No_Causa'], df_normalized['No_Efeito']))
edge_frequency = Counter(edge_list)

# Filtrar arestas com frequência abaixo do limiar
filtered_edges = [edge for edge, freq in edge_frequency.items() if freq >= LIMIAR_FREQUENCY]

print(f"Total de arestas únicas canônicas: {len(filtered_edges)}")
print(f"Arestas mantidas após filtrar por frequência >= {LIMIAR_FREQUENCY}: {len(filtered_edges)}")

Total de arestas únicas canônicas: 45
Arestas mantidas após filtrar por frequência >= 1: 45


In [9]:
# Criar o grafo direcionado com NetworkX
G = nx.DiGraph()
for cause, effect in filtered_edges:
  count = edge_frequency[(cause, effect)]
  G.add_edge(cause, effect, value=count, title=f"Contagem: {count}")

while not nx.is_directed_acyclic_graph(G):
  try:
    cycle = nx.find_cycle(G, orientation='original')
    print(f"Ciclo encontrado: {cycle}. Removendo aresta para manter DAG.")
    edge_to_remove = min(cycle, key=lambda edge: G.get_edge_data(edge[0], edge[1])['value'])
    G.remove_edge(edge_to_remove[0], edge_to_remove[1])
    print(f"Aresta removida: {edge_to_remove}")
  except nx.NetworkXNoCycle:
    break

isolated_nodes = list(nx.isolates(G))
G.remove_nodes_from(isolated_nodes)
print(f"Rede final com {G.number_of_nodes()} nós e {G.number_of_edges()} arestas após remover nós isolados.")

Ciclo encontrado: [('Pressão Arterial (PA)', 'Consumo de Sal / Sódio', 'forward'), ('Consumo de Sal / Sódio', 'Pressão Arterial (PA)', 'forward')]. Removendo aresta para manter DAG.
Aresta removida: ('Pressão Arterial (PA)', 'Consumo de Sal / Sódio', 'forward')
Ciclo encontrado: [('Fator de Risco', 'Hipertensão Arterial (HAS)', 'forward'), ('Hipertensão Arterial (HAS)', 'Pressão Arterial (PA)', 'forward'), ('Pressão Arterial (PA)', 'Fator de Risco', 'forward')]. Removendo aresta para manter DAG.
Aresta removida: ('Fator de Risco', 'Hipertensão Arterial (HAS)', 'forward')
Ciclo encontrado: [('Fator de Risco', 'Doenças Cardiovasculares (DVC)', 'forward'), ('Doenças Cardiovasculares (DVC)', 'Pressão Arterial (PA)', 'forward'), ('Pressão Arterial (PA)', 'Fator de Risco', 'forward')]. Removendo aresta para manter DAG.
Aresta removida: ('Doenças Cardiovasculares (DVC)', 'Pressão Arterial (PA)', 'forward')
Ciclo encontrado: [('Fator de Risco', 'Obesidade / Sobrepeso', 'forward'), ('Obesidade 

In [10]:
# 5. VISUALIZAÇÃO DA REDE COM PYVIS
print("Gerando visualização da rede...")

# Criar um objeto de rede Pyvis
net = Network(height="800px", width="100%", bgcolor="#222222", font_color="white", notebook=True, directed=True)

# Adicionar os nós e arestas do grafo NetworkX
net.from_nx(G)

# Ajustar a física para uma melhor visualização
net.set_options("""
var options = {
  "nodes": {
    "borderWidth": 2,
    "scaling": {
      "min": 15,
      "max": 50,
      "label": { "min": 14, "max": 30, "drawThreshold": 12, "maxVisible": 30 }
    },
    "font": {
      "size": 14,
      "face": "Tahoma"
    }
  },
  "edges": {
    "color": {
      "inherit": "from"
    },
    "smooth": {
      "type": "continuous"
    },
    "scaling":{
      "min": 1,
      "max": 15
    }
  },
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -20000,
      "centralGravity": 0.1,
      "springLength": 250
    },
    "maxVelocity": 50,
    "minVelocity": 0.75,
    "stabilization": {
      "iterations": 250
    }
  }
}
""")

# Gerar o arquivo HTML
try:
    net.save_graph(OUTPUT_VISUAL)
    print(f"Visualização da rede salva em '{OUTPUT_VISUAL}'. Abra este arquivo em seu navegador.")
except Exception as e:
    print(f"Erro ao salvar o gráfico: {e}")

Gerando visualização da rede...
Visualização da rede salva em 'bayesian_network.html'. Abra este arquivo em seu navegador.
